In [1]:
from riptide import *

In [2]:
# Load heart model
iCardio = cobra.io.load_matlab_model('/home/mjenior/Desktop/iCardio/iRno_HeartModel.mat')

In [3]:
iCardio

Name,iCardio_Rno
Memory address,0x07fb028dfbcc0
Number of metabolites,2886
Number of reactions,4183
Number of groups,0
Objective expression,1.0*RCR11017 - 1.0*RCR11017_reverse_8d3a1
Compartments,"c, r, s, m, p, g, l, n"


In [3]:
# Constrain media condition - 
exchanges = ['RCR30315','RCR90123','RCR30024','RCR30046','RCR30045','RCR30032','RCR30013','RCR30015','RCR30014','RCR30145','RCR30095','RCR30005','RCR30321','RCR30116','RCR30527','RCR30528','RCR30529','RCR30530','RCR30531','RCR90202','RCR30184','RCR30117','RCR30042','RCR30317','RCR30318','RCR30031','RCR30185']

for rxn in iCardio.reactions:
    if len(rxn.products) == 0:
        if not rxn.id in exchanges:
            rxn.bounds = (0., min([rxn.upper_bound, 1000.]))
        else:
            rxn.bounds = (max([rxn.lower_bound, -1000.]), min([rxn.upper_bound, 1000.]))

In [5]:
print('Genes: ' + str(len(iCardio.genes)))
print('Objective value: ' + str(iCardio.slim_optimize()))

Genes: 1728
Objective value: 761.7658939393942


In [4]:
# Read in Doxycycline 24 hour transcriptomic data and run RIPTiDe
dox_24 = riptide.read_transcription_file(file='/home/mjenior/Desktop/iCardio/Dox_24.tsv', 
                                         header=True, replicates=True)
iCardio_dox_24 = riptide.contextualize(iCardio, transcriptome=dox_24)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 112 from 4183 (97.32% change)
Metabolites pruned to 126 from 2886 (95.63% change)
Flux through the objective DECREASED to ~630.76 from ~761.77 (17.2% change)
Context-specific metabolism correlates with transcriptome (p=0.012 *)

RIPTiDe completed in 4 minutes and 2 seconds



In [6]:
# Read in DMSO 24 hour transcriptomic data and run RIPTiDe (second replicate)
dmso_24_2 = riptide.read_transcription_file(file='/home/mjenior/Desktop/iCardio/DMSO2_24.tsv', 
                                            header=True, replicates=True)
iCardio_dmso_24_2 = riptide.contextualize(iCardio, transcriptome=dmso_24_2)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 112 from 4183 (97.32% change)
Metabolites pruned to 126 from 2886 (95.63% change)
Flux through the objective DECREASED to ~630.76 from ~761.77 (17.2% change)
Context-specific metabolism correlates with transcriptome (p=0.014 *)

RIPTiDe completed in 4 minutes and 30 seconds



In [7]:
# Compare resultant model topology

test1 = set([x.id for x in iCardio_dox_24.model.genes])
test2 = set([x.id for x in iCardio_dmso_24_2.model.genes])

dox_only_gene = list(test1.difference(test2))
dmso_only_gene = list(test2.difference(test1))
core_gene = list(test1.intersection(test2))

print('Dox 24 hr only genes: ' + str(len(dox_only_gene)))
print('DMSO 24 hr only genes: ' + str(len(dmso_only_gene)) + '\n')

#--------------------------------------------------------------------------#

test1 = set([x.id for x in iCardio_dox_24.model.reactions])
test2 = set([x.id for x in iCardio_dmso_24_2.model.reactions])

dox_only_rxn = list(test1.difference(test2))
dmso_only_rxn = list(test2.difference(test1))
core_rxn = list(test1.intersection(test2))

print('Dox 24 hr only reactions: ' + str(len(dox_only_rxn)))
print('DMSO 24 hr only reactions: ' + str(len(dmso_only_rxn)) + '\n')

#--------------------------------------------------------------------------#

test1 = set([x.id for x in iCardio_dox_24.model.metabolites])
test2 = set([x.id for x in iCardio_dmso_24_2.model.metabolites])

dox_only_cpd = list(test1.difference(test2))
dmso_only_cpd = list(test2.difference(test1))
core_cpd = list(test1.intersection(test2))

print('Dox 24 hr only metabolites: ' + str(len(dox_only_cpd)))
print('DMSO 24 hr only metabolites: ' + str(len(dmso_only_cpd)) + '\n')

#--------------------------------------------------------------------------#

print('Shared genes: ' + str(len(core_gene)))
print('Shared reactions: ' + str(len(core_rxn)))
print('Shared metabolites: ' + str(len(core_cpd)))

Dox 24 hr only genes: 1
DMSO 24 hr only genes: 4

Dox 24 hr only reactions: 4
DMSO 24 hr only reactions: 4

Dox 24 hr only metabolites: 1
DMSO 24 hr only metabolites: 1

Shared genes: 255
Shared reactions: 108
Shared metabolites: 125


In [7]:
# Identify context-specific metabolites
compartments = {'c': 'cytoplasm', 
                'r': 'endoplasmic reticulum', 
                's': 'extracellular space', 
                'm': 'mitochondria', 
                'p': 'peroxisome', 
                'g': 'golgi', 
                'l': 'lysosome', 
                'n': 'nucleus'}


print('Dox 24 hr only metabolites:')
for x in dox_only_cpd:
    name = iCardio_dox_24.model.metabolites.get_by_id(x).name
    compartment = compartments[iCardio_dox_24.model.metabolites.get_by_id(x).compartment]
    print('\t' + name + ' from the ' + compartment)
    
print('\nDMSO 24 hr only metabolites:')
for y in dmso_only_cpd:
    name = iCardio_dmso_24_2.model.metabolites.get_by_id(y).name
    compartment = compartments[iCardio_dmso_24_2.model.metabolites.get_by_id(y).compartment]
    print('\t' + name + ' from the ' + compartment)

Dox 24 hr only metabolites:
	2-oxobutyrate from the mitochondria

DMSO 24 hr only metabolites:
	glucosamine-6-phosphate from the cytoplasm


In [8]:
# Save flux samples to file
iCardio_dox_24.flux_samples.to_csv('/home/mjenior/Desktop/iCardio/dox_24_fluxes.tsv', sep='\t')
iCardio_dmso_24_2.flux_samples.to_csv('/home/mjenior/Desktop/iCardio/dmso_24_fluxes.tsv', sep='\t')